# Python test

I'm using a Jupyter Nootbook for ease of sharing and nice display features. Obviously the below code can be pasted into a Python module and accessed through other interfaces.

In [5]:
import requests

# get users
users_download = requests.get('https://gist.githubusercontent.com/benjambles/ea36b76bc5d8ff09a51def54f6ebd0cb/raw/524e40ec297353b8070ff10ee0d9d847e44210f5/users.json')
users = eval(users_download.text)
# print(users)

# get venues
venues_download = requests.get('https://gist.githubusercontent.com/benjambles/ea36b76bc5d8ff09a51def54f6ebd0cb/raw/524e40ec297353b8070ff10ee0d9d847e44210f5/venues.json')
venues = eval(venues_download.text)
# print(venues)

places_to_go = []
places_to_avoid = []

# loop through venues
for venue in venues:
#     print(venue["name"])
    reasons = []
    
    # loop through users
    for user in users:
        # make foods and drinks all lowercase for comparison
        venue_foods = [x.lower() for x in venue["food"]]
        user_foods = [x.lower() for x in user["wont_eat"]]
        venue_drinks = [x.lower() for x in venue["drinks"]]
        user_drinks = [x.lower() for x in user["drinks"]]
        
        # list people who cannot eat any food
#         print("compare {0} to {1} for {2}".format(venue["food"], user["wont_eat"], user["name"]))
        rejected_foods = set(venue_foods).intersection(user_foods)
        if set(venue_foods) == rejected_foods:
            reason = "There is nothing for {0} to eat".format(user["name"])
#             print("{0} - see {1} vs {2}".format(reason, venue["food"], user["wont_eat"]))
            reasons.append(reason)
        
        # list people who cannot find a drink they like
#         print("compare {0} to {1} for {2}".format(venue["drinks"], user["drinks"], user["name"]))
        selected_drinks = set(venue_drinks).intersection(user_drinks)
        if not selected_drinks:
            reason = "There is nothing for {0} to drink".format(user["name"])
#             print("{0} - see {1} vs {2}".format(reason, venue["drinks"], user["drinks"]))
            reasons.append(reason)
        
    
    # if any reason not to go, add to places to avoid
    if reasons:
        item = {
            "Name": venue["name"],
            "Reasons": reasons,
        }
        places_to_avoid.append(item)
    # otherwise, add to places to go
    else:
        item = {
            "Name": venue["name"],
        }
        places_to_go.append(item)
        
# display places to go
print("\nPlaces to go:")
for place in places_to_go:
    print "* {0}".format(place["Name"])

# display places to avoid
print("\nPlaces to avoid:")
for place in places_to_avoid:
    print "* {0}".format(place["Name"])
    for reason in place["Reasons"]:
        print "  * {0}".format(reason)



Places to go:
* Spice of life
* The Cambridge

Places to avoid:
* El Cantina
  * There is nothing for Robert Webb to drink
  * There is nothing for Bobby Robson to eat
* Twin Dynasty
  * There is nothing for David Lang to eat
* Wagamama
  * There is nothing for Robert Webb to drink
* Sultan Sofrasi
  * There is nothing for Robert Webb to drink
* Spirit House
  * There is nothing for Alan Allen to drink
* Tally Joe
  * There is nothing for Robert Webb to drink
* Fabrique
  * There is nothing for Robert Webb to drink
  * There is nothing for David Lang to drink


# SQL test

I opted for a SQLLite solution here to keep it simple and portable. See the workings below and then answers at the end...

In [2]:
import sqlite3
db = sqlite3.connect(':memory:')

# create prices table
cursor = db.cursor()
cursor.execute('''
CREATE TABLE prices(product TEXT, price_effective_date DATE, price INT)
''')
db.commit()

# populate prices table
prices = [("product_1", "01/01/2018", 50),
("product_2", "01/01/2018", 40),
("product_1", "03/01/2018", 25),
("product_2", "05/01/2018", 20),
("product_1", "10/01/2018", 50),
("product_2", "12/01/2018", 40)]
cursor.executemany('''
INSERT INTO prices(product, price_effective_date, price) VALUES(?,?,?)
''', prices)
db.commit()

# create sales table
cursor = db.cursor()
cursor.execute('''
CREATE TABLE sales(product TEXT, sales_date DATE, quantity INT)
''')
db.commit()

# populate sales table table
sales = [("product_1", "01/01/2018", 10),
("product_2", "02/01/2018", 12),
("product_1", "04/01/2018", 50),
("product_2", "06/01/2018", 70),
("product_1", "12/01/2018", 8),
("product_2", "15/01/2018", 9)]
cursor.executemany('''
INSERT INTO sales(product, sales_date, quantity) VALUES(?,?,?)
''', sales)
db.commit()

# calculate total revenue
total_sql = """
SELECT SUM(p.price * s.quantity)
FROM sales s
INNER JOIN (
SELECT product, price_effective_date, price
, COALESCE(MIN(next_price_effective_date),'99/99/9999') AS next_price_effective_date
FROM (
SELECT p1.product, p1.price_effective_date, p1.price
, p2.price_effective_date AS next_price_effective_date
FROM prices p1
LEFT OUTER JOIN prices p2
ON p1.product = p2.product
AND p1.price_effective_date < p2.price_effective_date
ORDER by p1.product, p1.price_effective_date
) x
GROUP BY product, price_effective_date, price
) p
ON s.product = p.product
AND s.sales_date >= p.price_effective_date
AND s.sales_date < p.next_price_effective_date
"""
cursor.execute(total_sql)
total_result = cursor.fetchone()

db.close()

## Question 1

In [7]:
print(total_result)

(4390,)


## Question 2

In [4]:
print(total_sql)


SELECT SUM(p.price * s.quantity)
FROM sales s
INNER JOIN (
SELECT product, price_effective_date, price
, COALESCE(MIN(next_price_effective_date),'99/99/9999') AS next_price_effective_date
FROM (
SELECT p1.product, p1.price_effective_date, p1.price
, p2.price_effective_date AS next_price_effective_date
FROM prices p1
LEFT OUTER JOIN prices p2
ON p1.product = p2.product
AND p1.price_effective_date < p2.price_effective_date
ORDER by p1.product, p1.price_effective_date
) x
GROUP BY product, price_effective_date, price
) p
ON s.product = p.product
AND s.sales_date >= p.price_effective_date
AND s.sales_date < p.next_price_effective_date

